In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

In [18]:
def get_weather(city: str) -> str:
    """Get weather of given city
    Args:
        city: city to get weather for
    Returns:
        Weather of the city
    """
    return "Sunny"

tools = [get_weather]

In [19]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import AIMessage
from langgraph.types import interrupt

checkpoint = InMemorySaver()
RISKY_TOOLS = ["get_weather"]

def halt_on_risky_tools(state):
    lastmsg = state["messages"][-1]
    if isinstance(lastmsg, AIMessage) and getattr(lastmsg, "tool_calls", None):
        for tc in lastmsg.tool_calls:
            if tc.get("name") in RISKY_TOOLS:
                feedback = interrupt({"awaiting": tc["name"], "args": tc.get("args", {})})
                print(f"feedbacl:{feedback}")
    return {}

agent = create_react_agent(model=llm, tools=tools, checkpointer=checkpoint, post_model_hook=halt_on_risky_tools)

In [ ]:
from langgraph.types import Command

for event in agent.stream(Command(resume=True), config={"thread_id": 1}):
    print(event)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "Osaka"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--4e01712b-3b6a-410d-bf50-5d6e4b3d343f-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Osaka'}, 'id': 'f33c753e-eb61-4047-a484-459e46216240', 'type': 'tool_call'}], usage_metadata={'input_tokens': 68, 'output_tokens': 15, 'total_tokens': 132, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 49}})]}}
{'__interrupt__': (Interrupt(value={'awaiting': 'get_weather', 'args': {'city': 'Osaka'}}, id='875cbb0991aa9c350e3036cff05b958e'),)}
